In [1]:
def egcd(a, b):
    x0, x1, y0, y1 = 0, 1, 1, 0
    while a != 0:
        #(q, a), b = divmod(b, a), a
        q, a, b = b // a, b % a, a
        y0, y1 = y1, y0 - q * y1
        x0, x1 = x1, x0 - q * x1
    return b, x0, y0

def gcd(x, y):
    res = 0
    if x > y:
        small = y
    else:
        small = x
    for i in range(1, small + 1):
        if (x % i == 0) and (y % i == 0):
            res = i

    return res

import secrets
import string


def genrandomstring(length):
    password_characters = string.ascii_letters + string.digits + string.punctuation + string.whitespace
    password = ''.join(secrets.choice(password_characters) for i in range(length))
    return password

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m
    
def modpow(a, n, p):
    res = 1
    a = a % p

    while n > 0:
        if n % 2:
            res = (res * a) % p
            n = n - 1
        else:
            a = (a ** 2) % p
            n = n // 2

    return res % p


def getrandom(size):
    prime = secrets.randbits(size)
    return prime


def randomexponent(phi):
    e = 0
    good = False
    while not good:
        e = getrandom(16)
        e = e % 19193
        e += 46341
        e += 1 - e % 2
        if e > 65535:
            e = 46341
        while gcd(e, phi) != 1:
            e = e + 2
        if e <= phi:
            good = True
    return e



In [2]:
def fermatprimalitytest(n, k):
    if n == 1 or n == 4:
        return False
    if n == 2 or n == 3:
        return True

    if n % 2 == 0:
        return False

    for i in range(k):
        a = secrets.randbelow(n - 1)
        if a < 1:
            a = 1

        #   if gcd(n, a) != 1:
            #   return False

        if modpow(a, n - 1, n) != 1:
            return False

    return True

def millerwitness(d, n):
    if n < 4:
        return False
    a = secrets.randbelow(n - 2)
    if a < 2 and n > 3:
        a = a + 2
    if a < 1:
        a = 1

    x = modpow(a, d, n)

    if x == 1 or x == n - 1:
        return True

    while d != n - 1:
        x = (x * x) % n
        d *= 2

        if x == 1:
            return False
        if x == n - 1:
            return True
    return False



def millerrabinprimalitytest(n, k):
    # Corner cases
    if n <= 1 or n == 4:
        return False
    if n <= 3:
        return True

    # Find r such that n = 2^d * r + 1 for some r >= 1
    d = n - 1
    while d % 2 == 0:
        d //= 2

    for i in range(k):
        if not millerwitness(d, n):
            return False

    return True



def randomprime():
    # random number between 46341 and 65535
    prime = getrandom(16)
    prime = prime % 19193
    prime += 46341
    prime += 1 - prime % 2
    if prime > 65521:
        prime = 65521
    while not fermatprimalitytest(prime, 40) or not millerrabinprimalitytest(prime, 40):
        prime = prime + 2
    return prime


def crt(items):
    # Determine N, the product of all n_i
    N = 1
    for a, n in items:
        N *= n

    # Find the solution (mod N)
    result = 0
    for a, n in items:
        m = N // n
        d, r, s = egcd(n, m)
        if d != 1:
            raise Exception("Input not pairwise co-prime")
        result += a * s * m

    return result % N

In [3]:
def rsaencrypt(plaintxt, pubkey, isint):
    if isint:
        cipher = [modpow(char, pubkey.exp, pubkey.mod) for char in plaintxt]
    else:
        cipher = [modpow(ord(char), pubkey.exp, pubkey.mod) for char in plaintxt]

    return cipher

In [4]:
class PublicKey:
    def __init__(self, exp, mod):
        self.exp = exp
        self.mod = mod
        
class PrivateKey:
    def __init__(self, modinv, mod):
        self.modinv = modinv
        self.mod = mod

In [7]:
def hastadbroadcast():
    plaintxt = genrandomstring(128)
    good = False
    mod1, mod2, mod3 = 0, 0, 0
    while good is not True:
        p1 = randomprime()
        p2 = randomprime()
        mod1 = p1 * p2
        p3 = randomprime()
        p4 = randomprime()
        mod2 = p3 * p4
        p5 = randomprime()
        p6 = randomprime()
        mod3 = p5 * p6
        print("random prime 1 numbers are", p1, "and", p2, '\n')
        print("random prime 2 numbers are", p3, "and", p4, '\n')
        print("random prime 3 numbers are", p5, "and", p6, '\n')
        print("modulo 1 is", mod1, '\n')
        print("modulo 2 is", mod2, '\n')
        print("modulo 3 is", mod3, '\n')
        good = True
        if egcd(mod1, mod2)[0] != 1:
            print("mod1 & mod2 aren't coprime")
            good = False
        if egcd(mod1, mod3)[0] != 1:
            print("mod1 & mod3 aren't coprime")
            good = False
        if egcd(mod2, mod3)[0] != 1:
            print("mod2 & mod3 aren't coprime")
            good = False
    e = 3
    print("exponent is", mod1, '\n')
    c1 = rsaencrypt(plaintxt, PublicKey(e, mod1), False)
    c2 = rsaencrypt(plaintxt, PublicKey(e, mod2), False)
    c3 = rsaencrypt(plaintxt, PublicKey(e, mod3), False)
    print("Plain text START")
    print(plaintxt)
    print("Plain text END\n")
    print("Cipher text 1 = ", c1, '\n')
    print("Cipher text 2 = ", c2, '\n')
    print("Cipher text 3 = ", c3, '\n')
    result = ""
    for i in range(0, len(c1)):
        crtres = crt([(c1[i], mod1), (c2[i], mod2), (c3[i], mod3)])
        print("[ATTACK] crt result =", crtres)
        crtroot = round(crtres ** (1 / e))
        print("[ATTACK] racine cubique =", crtroot)
        result = result + chr(round(crtroot))
    print("Deciphered text START")
    print(result)
    print("Deciphered text END\n")

In [8]:
hastadbroadcast()

random prime 1 numbers are 58657 and 58147 

random prime 2 numbers are 54601 and 58511 

random prime 3 numbers are 51613 and 60679 

modulo 1 is 3410728579 

modulo 2 is 3194759111 

modulo 3 is 3131825227 

exponent is 3410728579 

Plain text START

5P6/76U+Ck1F7!`
Plain text END

Cipher text 1 =  [1000, 1225043, 1771561, 1331000, 1000000, 39304, 185193, 1295029, 50653, 59319, 941192, 1728, 1860867, 103823, 830584, 148877, 175616, 658503, 681472, 91125, 474552, 1092727, 74088, 1728000, 1092727, 1728, 1520875, 474552, 35937, 46656, 729000, 59319, 1771561, 2000376, 857375, 132651, 438976, 704969, 262144, 117649, 97336, 2197, 493039, 148877, 148877, 1295029, 438976, 1259712, 421875, 941192, 405224, 140608, 753571, 456533, 551368, 328509, 103823, 1061208, 185193, 405224, 39304, 421875, 54872, 91125, 658503, 91125, 830584, 857375, 531441, 571787, 140608, 1157625, 314432, 658503, 658503, 46656, 54872, 2000376, 1728, 103823, 373248, 1685159, 1124864, 373248, 68921, 110592, 1295029, 1728, 5